In [1]:
import os
import json

parent_dir = 'supervised/supervised/'
file_list = [os.path.join(parent_dir, file) for file in os.listdir(parent_dir)]

In [2]:
raw_lines = []
for file in file_list:
    with open(file, encoding='utf-8') as f:
        lines = f.readlines()
    lines[-1] = lines[-1] + '\n'
    lines.extend(['\n'])
    lines = [line.strip() for line in lines]
    raw_lines.extend(lines)

all_sentences = []
beg, end = 0, 0
while end < len(raw_lines):
        if raw_lines[end]:
            end += 1
        else:
            sentence = raw_lines[beg:end]
            all_sentences.append(sentence)
            end += 1
            beg = end
print(len(all_sentences))

188239


In [3]:
washed_sentences = set()
no_entities_sentences = set()
for sentence in all_sentences:
    O_count = sum([1 if wt.endswith('\tO') else 0 for wt in sentence])
    if O_count != len(sentence):
        washed_sentences.add('\n'.join(sentence))
    else:
        no_entities_sentences.add('\n'.join(sentence))
print(len(washed_sentences))
print(len(no_entities_sentences))
all_sentences = list(washed_sentences)

154387
23482


In [4]:
coarse_classes = set()
fine_classes = set()
for k in range(len(all_sentences)):
    wts = all_sentences[k].split('\n')
    for i in range(len(wts)):
        word_tag = wts[i]
        tokens = word_tag.split('\t')
        if tokens[1] != 'O':
            if '/' in tokens[1]:
                tokens[1] = tokens[1].replace('/', ',')
                wts[i] = '\t'.join(tokens)
            fine_classes.add(tokens[1])
            coarse_classes.add(tokens[1].split('-')[0])
    all_sentences[k] = '\n'.join(wts)
print(f'coarse-grained classes: {coarse_classes}; \nfine-grained classes: {fine_classes}')

coarse_dict = {}
fine_dict = {}

coarse_sid_dict = {}
fine_sid_dict = {}

coarse_sentences = ['\n'.join([wt.replace(wt.split('\t')[1], wt.split('\t')[1].split('-')[0]) for wt in sentence.split('\n')]) for sentence in all_sentences]
print(coarse_sentences[0])

for coarse_class in coarse_classes:
    sentences = []
    sid_list = []
    out_sid = 0
    for sentence in coarse_sentences:
        wts = sentence.split('\n')
        for word_tag in wts:
            if word_tag.endswith(coarse_class):
                sentences.append(sentence)
                sid_list.append(out_sid)
                break
        out_sid += 1
    coarse_dict[coarse_class] = sentences
    coarse_sid_dict[coarse_class] = sid_list

for fine_class in fine_classes:
    sentences = []
    sid_list = []
    out_sid = 0
    for sentence in all_sentences:
        wts = sentence.split('\n')
        for word_tag in wts:
            if word_tag.endswith(fine_class):
                sentences.append(sentence)
                sid_list.append(out_sid)
                break
        out_sid += 1
    fine_dict[fine_class] = sentences
    fine_sid_dict[fine_class] = sid_list

coarse-grained classes: {'person', 'building', 'event', 'other', 'product', 'location', 'art', 'organization'}; 
fine-grained classes: {'building-hospital', 'art-broadcastprogram', 'other-disease', 'building-restaurant', 'location-road,railway,highway,transit', 'other-currency', 'event-disaster', 'organization-education', 'other-chemicalthing', 'building-airport', 'event-other', 'art-other', 'location-park', 'other-language', 'product-food', 'location-GPE', 'other-educationaldegree', 'building-other', 'building-theater', 'product-ship', 'art-writtenart', 'organization-sportsteam', 'art-painting', 'other-livingthing', 'event-attack,battle,war,militaryconflict', 'product-weapon', 'building-hotel', 'organization-showorganization', 'location-other', 'organization-company', 'other-law', 'organization-politicalparty', 'product-other', 'location-island', 'location-mountain', 'product-software', 'other-award', 'person-artist,author', 'other-medical', 'person-actor', 'other-god', 'organization-

In [5]:
# remove duplicated sentences among different classes
key_list = list(coarse_sid_dict.keys())
for i in range(len(key_list)):
    a_sids = set(coarse_sid_dict[key_list[i]])
    for j in range(i + 1, len(key_list)):
        b_sids = set(coarse_sid_dict[key_list[j]])
        c = a_sids.intersection(b_sids)
        m, n = len(a_sids), len(b_sids)
        for sid in c:
            removed_sentence = coarse_sentences[sid]
            if m > n:
                coarse_dict[key_list[i]].remove(removed_sentence)
                coarse_sid_dict[key_list[i]].remove(sid)
                a_sids.remove(sid)
                m -= 1
            else:
                coarse_dict[key_list[j]].remove(removed_sentence)
                coarse_sid_dict[key_list[j]].remove(sid)
                b_sids.remove(sid)
                n -= 1

total_sentence_num = 0

# remove duplicated sentences among different classes
key_list = list(fine_sid_dict.keys())
for i in range(len(key_list)):
    a_sids = set(fine_sid_dict[key_list[i]])
    for j in range(i + 1, len(key_list)):
        b_sids = set(fine_sid_dict[key_list[j]])
        c = a_sids.intersection(b_sids)
        m, n = len(a_sids), len(b_sids)
        for sid in c:
            removed_sentence = all_sentences[sid]
            if m > n:
                fine_dict[key_list[i]].remove(removed_sentence)
                fine_sid_dict[key_list[i]].remove(sid)
                a_sids.remove(sid)
                m -= 1
            else:
                fine_dict[key_list[j]].remove(removed_sentence)
                fine_sid_dict[key_list[j]].remove(sid)
                b_sids.remove(sid)
                n -= 1

coarse_tag_count, fine_tag_count = {}, {}
for c in coarse_classes:
    coarse_tag_count[c] = len(coarse_dict[c])
for c in fine_classes:
    fine_tag_count[c] = len(fine_dict[c])
print(coarse_tag_count)
print(fine_tag_count)

{'person': 34843, 'building': 14757, 'event': 14452, 'other': 20353, 'product': 14738, 'location': 23014, 'art': 12676, 'organization': 19554}
{'building-hospital': 1239, 'art-broadcastprogram': 1924, 'other-disease': 1959, 'building-restaurant': 770, 'location-road,railway,highway,transit': 3719, 'other-currency': 1978, 'event-disaster': 877, 'organization-education': 4152, 'other-chemicalthing': 2087, 'building-airport': 1280, 'event-other': 2452, 'art-other': 1591, 'location-park': 1557, 'other-language': 1927, 'product-food': 1044, 'location-GPE': 12243, 'other-educationaldegree': 1175, 'building-other': 4445, 'building-theater': 1612, 'product-ship': 1401, 'art-writtenart': 2180, 'organization-sportsteam': 3688, 'art-painting': 213, 'other-livingthing': 1908, 'event-attack,battle,war,militaryconflict': 2579, 'product-weapon': 1474, 'building-hotel': 1079, 'organization-showorganization': 1900, 'location-other': 3619, 'organization-company': 5214, 'other-law': 1776, 'organization-p

In [6]:
corse_distribution = {}
for c in coarse_dict:
    distribution = {}
    for s in coarse_dict[c]:
        t_set = set()
        for w_t in s.split('\n'):
            t = w_t.split('\t')[1] 
            if t != 'O':
                t_set.add(t.split('-')[0])
        for t in t_set:
            distribution[t] = distribution.get(t, 0) + 1
    corse_distribution[c] = distribution
with open('./continual/coarse/distribution.json', encoding='utf-8', mode='w') as f:
    json.dump(corse_distribution, f)

fine_distribution = {}
for c in fine_dict:
    distribution = {}
    for s in fine_dict[c]:
        t_set = set()
        for w_t in s.split('\n'):
            t = w_t.split('\t')[1] 
            #\ .split('-')[0]
            if t != 'O':
                t_set.add(t)
        for t in t_set:
            distribution[t] = distribution.get(t, 0) + 1
    fine_distribution[c] = distribution

with open('./continual/fine/distribution.json', encoding='utf-8', mode='w') as f:
    json.dump(fine_distribution, f)

In [7]:
del washed_sentences

In [8]:
import random

In [9]:
no_coarse_dict, no_fine_dict = {}, {}
for coarse in coarse_classes:
    sample_ids = coarse_sid_dict[coarse]
    no_samples = []
    for sample_id in sample_ids:
        sample = coarse_sentences[sample_id]
        wts = sample.split('\n')
        for i in range(len(wts)):
            word_tag = wts[i].split('\t')
            if word_tag[1] != 'O' and word_tag[1] != coarse:
                word_tag[1] = 'O'
                wts[i] = '\t'.join(word_tag)
        no_samples.append('\n'.join(wts))
    no_coarse_dict[coarse] = no_samples
for fine in fine_classes:
    sample_ids = fine_sid_dict[fine]
    no_samples = []
    for sample_id in sample_ids:
        sample = all_sentences[sample_id]
        wts = sample.split('\n')
        for i in range(len(wts)):
            word_tag = wts[i].split('\t')
            if word_tag[1] != 'O' and word_tag[1] != fine:
                word_tag[1] = 'O'
                wts[i] = '\t'.join(word_tag)
        no_samples.append('\n'.join(wts))
    no_fine_dict[fine] = no_samples

In [10]:
no_coarse_train_samples, no_coarse_valid_samples, no_coarse_test_samples = {}, {}, {}
# no_coarse_train_indices, no_coarse_valid_indices, no_coarse_test_indices = {}, {}, {}
no_fine_train_samples, no_fine_valid_samples, no_fine_test_samples = {}, {}, {}
# no_fine_train_indices, no_fine_valid_indices, no_fine_test_indices = {}, {}, {}

coarse_train_samples, coarse_valid_samples, coarse_test_samples = {}, {}, {}
# coarse_train_indices, coarse_valid_indices, coarse_test_indices = {}, {}, {}
fine_train_samples, fine_valid_samples, fine_test_samples = {}, {}, {}
# fine_train_indices, fine_valid_indices, fine_test_indices = {}, {}, {}


for c in coarse_classes:
    sample_ids = coarse_sid_dict[c]
    train_sample_id_indices, valid_sample_id_indices = set(), set()
    train_count, valid_count = 0, 0
    while train_count < 0.7 * coarse_tag_count[c]:
        sample_id_idx = random.choice(range(len(sample_ids)))
        if sample_id_idx not in train_sample_id_indices:
            train_sample_id_indices.add(sample_id_idx)
            train_count += 1
    while valid_count < 0.1 * coarse_tag_count[c]:
        sample_id_idx = random.choice(range(len(sample_ids)))
        if sample_id_idx not in train_sample_id_indices and sample_id_idx not in valid_sample_id_indices:
            valid_sample_id_indices.add(sample_id_idx)
            valid_count += 1
    test_sample_id_indices = set(range(len(sample_ids))).difference(train_sample_id_indices.union(valid_sample_id_indices))
    no_coarse_train_samples[c] = [no_coarse_dict[c][sample_id_idx] for sample_id_idx in train_sample_id_indices]
    coarse_train_samples[c] = [coarse_dict[c][sample_id_idx] for sample_id_idx in train_sample_id_indices]
    no_coarse_valid_samples[c] = [no_coarse_dict[c][sample_id_idx] for sample_id_idx in valid_sample_id_indices]
    coarse_valid_samples[c] = [coarse_dict[c][sample_id_idx] for sample_id_idx in valid_sample_id_indices]
    no_coarse_test_samples[c] = [no_coarse_dict[c][sample_id_idx] for sample_id_idx in test_sample_id_indices]
    coarse_test_samples[c] = [coarse_dict[c][sample_id_idx] for sample_id_idx in test_sample_id_indices]
    
#     no_coarse_train_indices[c] = train_sample_id_indices
#     no_coarse_valid_indices[c] = valid_sample_id_indices
#     no_coarse_test_indices[c] = test_sample_id_indices
for c in coarse_tag_count:
    print(f'{c} : train : {len(no_coarse_train_samples[c])}; valid: {len(no_coarse_valid_samples[c])}; test : {len(no_coarse_test_samples[c])}; total : {coarse_tag_count[c]}; ratio : {len(no_coarse_train_samples[c]) / coarse_tag_count[c]}')

    
print('================================================')

    
for c in fine_classes:
    sample_ids = fine_sid_dict[c]
    train_sample_id_indices, valid_sample_id_indices = set(), set()
    train_count, valid_count = 0, 0
    while train_count < 0.7 * fine_tag_count[c]:
        sample_id_idx = random.choice(range(len(sample_ids)))
        if sample_id_idx not in train_sample_id_indices:
            train_sample_id_indices.add(sample_id_idx)
            train_count += 1
    while valid_count < 0.1 * fine_tag_count[c]:
        sample_id_idx = random.choice(range(len(sample_ids)))
        if sample_id_idx not in train_sample_id_indices and sample_id_idx not in valid_sample_id_indices:
            valid_sample_id_indices.add(sample_id_idx)
            valid_count += 1
    test_sample_id_indices = set(range(len(sample_ids))).difference(train_sample_id_indices.union(valid_sample_id_indices))
    no_fine_train_samples[c] = [no_fine_dict[c][sample_id_idx] for sample_id_idx in train_sample_id_indices]
    no_fine_valid_samples[c] = [no_fine_dict[c][sample_id_idx] for sample_id_idx in valid_sample_id_indices]
    no_fine_test_samples[c] = [no_fine_dict[c][sample_id_idx] for sample_id_idx in test_sample_id_indices]
    
    fine_train_samples[c] = [fine_dict[c][sample_id_idx] for sample_id_idx in train_sample_id_indices]
    fine_valid_samples[c] = [fine_dict[c][sample_id_idx] for sample_id_idx in valid_sample_id_indices]
    fine_test_samples[c] = [fine_dict[c][sample_id_idx] for sample_id_idx in test_sample_id_indices]
#     no_fine_train_indices[c] = train_sample_id_indices
#     no_fine_valid_indices[c] = valid_sample_id_indices
#     no_fine_test_indices[c] = test_sample_id_indices
for c in fine_tag_count:
    print(f'{c} : train : {len(no_fine_train_samples[c])}; valid: {len(no_fine_valid_samples[c])}; test : {len(no_fine_test_samples[c])}; total : {fine_tag_count[c]}; ratio : {len(no_fine_train_samples[c]) / fine_tag_count[c]}')

person : train : 24391; valid: 3485; test : 6967; total : 34843; ratio : 0.7000258301523979
building : train : 10330; valid: 1476; test : 2951; total : 14757; ratio : 0.7000067764450769
event : train : 10117; valid: 1446; test : 2889; total : 14452; ratio : 0.7000415167450872
other : train : 14248; valid: 2036; test : 4069; total : 20353; ratio : 0.7000442195253771
product : train : 10317; valid: 1474; test : 2947; total : 14738; ratio : 0.7000271407246573
location : train : 16110; valid: 2302; test : 4602; total : 23014; ratio : 0.7000086903623881
art : train : 8874; valid: 1268; test : 2534; total : 12676; ratio : 0.7000631113916062
organization : train : 13688; valid: 1956; test : 3910; total : 19554; ratio : 0.7000102280863251
building-hospital : train : 868; valid: 124; test : 247; total : 1239; ratio : 0.7005649717514124
art-broadcastprogram : train : 1347; valid: 193; test : 384; total : 1924; ratio : 0.7001039501039501
other-disease : train : 1372; valid: 196; test : 391; total

In [ ]:
coarse_train_samples, coarse_test_samples = {}, {}
fine_train_samples, fine_test_samples = {}, {}
for c in coarse_classes:
    train_samples = [coarse_dict[c][i] for i in no_coarse_train_indices[c]]
    coarse_train_samples[c] = train_samples
for c in coarse_tag_count:
    print(f'{c} : train : {len(coarse_train_samples[c])}; total : {coarse_tag_count[c]}; ratio : {len(coarse_train_samples[c]) / coarse_tag_count[c]}')
# remained_samples = set()
for c in fine_classes:
    train_samples = [fine_dict[c][i] for i in no_fine_train_indices[c]]
    fine_train_samples[c] = train_samples
#     remained_samples = remained_samples.union(set([fine_dict[c][i] for i in no_fine_test_indices[c]]))
for c in fine_tag_count:
    print(f'{c} : train : {len(fine_train_samples[c])}; total : {fine_tag_count[c]}; ratio : {len(fine_train_samples[c]) / fine_tag_count[c]}')

# for c in fine_tag_count:
#     train_samples = fine_train_samples[c]
#     train_dist = {}
#     for train_sample in train_samples:
#         tags = set(wt .split('\t')[1] for wt in train_sample.split('\n'))
#         tags.remove('O')
#         for tag in tags:
#             train_dist[tag] = train_dist.get(tag, 0) + 1
#     test_samples, test_tag_count = set(), {}
#     for train_tag in train_dist:
#         visited = set()
#         while test_tag_count.get(train_tag, 0) < (0.25 * train_dist[train_tag] if train_dist[train_tag] > 4 else 1):
#             test_sample = random.choice(list(remained_samples))
#             visited.add(test_sample)
#             if visited == remained_samples:
#                 print('No satisfied sample. Exit search.')
#                 break
#             if test_sample not in test_samples and test_sample not in train_samples:
#                 satisfied = True
#                 tags = set([wt.split('\t')[1] for wt in test_sample.split('\n')])
#                 tags.remove('O')
#                 if tags.issubset(train_dist.keys()):
#                     for t in tags:
#                         if test_tag_count.get(t, 0) + 1 > (0.25 * train_dist[t] if train_dist[t] > 4 else 1):
#                             satisfied = False
#                             break
#                 else:
#                     satisfied = False
#                 if satisfied:
#                     test_samples.add(test_sample)
#                     remained_samples.remove(test_sample)
#                     visited.remove(test_sample)
#                     test_tag_count[train_tag] = test_tag_count.get(train_tag, 0) + 1
#                     for t in tags:
#                         if t != 'O' and t != train_tag:
#                             test_tag_count[t] = test_tag_count.get(t, 0) + 1
#     fine_test_samples[c] = test_samples
#     print(f'{c} test samples found!')
# for c in tag_count:
#     print(f'{c} : test : {len(fine_test_samples[c])}; total : {fine_tag_count[c]}; ratio : {len(fine_test_samples[c]) / fine_tag_count[c]}')



In [13]:
def save_file(parent_dir, samples_dict, mode):
    for fname in samples_dict:
        path = os.path.join(parent_dir, fname)
        if not os.path.exists(path):
            os.makedirs(path)
        with open(os.path.join(path, mode+'.txt'), 'w', encoding='utf-8') as f:
            f.writelines('\n\n'.join(samples_dict[fname]))
save_file('./continual/coarse/disjoint', no_coarse_train_samples, mode='train')
save_file('./continual/coarse/disjoint', no_coarse_valid_samples, mode='dev')
save_file('./continual/coarse/disjoint', no_coarse_test_samples, mode='test')
save_file('./continual/fine/disjoint', no_fine_train_samples, mode='train')
save_file('./continual/fine/disjoint', no_fine_valid_samples, mode='dev')
save_file('./continual/fine/disjoint', no_fine_test_samples, mode='test')

save_file('./continual/coarse/joint', coarse_train_samples, mode='train')
save_file('./continual/coarse/joint', coarse_valid_samples, mode='dev')
save_file('./continual/coarse/joint', coarse_test_samples, mode='test')
save_file('./continual/fine/joint', fine_train_samples, mode='train')
save_file('./continual/fine/joint', fine_valid_samples, mode='dev')
save_file('./continual/fine/joint', fine_test_samples, mode='test')

save_file('./continual/fine/joint', fine_test_samples, mode='test')

# save_file('./continual/fine/overlapping', fine_test_samples, mode='test')

In [ ]:
with open(os.path.join('no_entities.txt'), 'w', encoding='utf-8') as f:
    f.writelines('\n\n'.join(no_entities_sentences))